In [12]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [13]:
# importing the dataset
train_df = pd.read_csv('labeledTrainData.tsv',header=0, delimiter = "\t", quoting = 3)
test_df = pd.read_csv('testData.tsv',header=0, delimiter = "\t", quoting = 3)


In [14]:
#Cleaning text of trainig set
from bs4 import BeautifulSoup  #Removing HTML Markup: The BeautifulSoup Package
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []

for i in tqdm(range(0,25000)):
    review = BeautifulSoup(train_df['review'][i])
    review = re.sub('[^a-zA-Z]', ' ', train_df['review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)


  0%|                                                                                        | 0/25000 [00:00<?, ?it/s]C:\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|████████████████████████████████████████████████████████████████████████████| 25000/25000 [36:28<00:00, 11.42it/s]


In [15]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
X_train = cv.fit_transform(corpus).toarray()
Y_train = train_df.iloc[:,1].values

In [16]:
# Fitting RandomForest to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100)
classifier = classifier.fit(X_train, Y_train)


In [17]:
#Cleaning text of test set
corpus2 = []

for i in tqdm(range(0,25000)):
    review = BeautifulSoup(test_df['review'][i])
    review = re.sub('[^a-zA-Z]', ' ', test_df['review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus2.append(review)



  0%|                                                                                        | 0/25000 [00:00<?, ?it/s]C:\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
100%|████████████████████████████████████████████████████████████████████████████| 25000/25000 [36:53<00:00, 11.29it/s]


In [18]:
# Get a bag of words for the test set, and convert to a numpy array
X_test = cv.transform(corpus2).toarray()

In [19]:
# Fitting RandomForest to the Test set
result = classifier.predict(X_test)


In [20]:
# Copy the results to a pandas dataframe with an "id" column and a "sentiment" column
output = pd.DataFrame( data={"id":test_df["id"], "sentiment":result} )

In [21]:
# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )